In [ ]:
from math import trunc
from select import KQ_NOTE_WRITE

import polars as pl
import polars.selectors as cs

import altair as alt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [ ]:
df_path = r'/Users/zygimantas/Downloads/computer_prices_all.csv'

In [ ]:
df = pl.read_csv(df_path)

In [ ]:
df.describe()

In [ ]:
df.collect_schema()

In [ ]:
df.select(
    pl.col('brand')
).group_by(
    'brand'
).len().sort(
    by='len',
    descending=True
)

In [ ]:
df.head(5)

In [ ]:
df = df.with_columns(
    pl.col('resolution').str.split('x').list.get(0).alias('resolution_width').cast(pl.Int16),
    pl.col('resolution').str.split('x').list.get(1).alias('resolution_height').cast(pl.Int16)
)

In [ ]:
df = df.with_columns(
    (datetime.now().year - pl.col('release_year')).alias('device_age')
)

In [ ]:
df = df.with_columns(
    pl.when((pl.col('device_type') == 'Desktop') & (pl.col('battery_wh') == 0))
    .then(pl.lit(None))
    .otherwise(pl.col('battery_wh'))
    .alias('battery_wh'),
    pl.when((pl.col('device_type') == 'Laptop') & (pl.col('psu_watts') == 0))
    .then(pl.lit(None))
    .otherwise(pl.col('psu_watts'))
    .alias('psu_watts')
)

In [ ]:
df = df.with_columns(
    pl.col('cpu_model'),
    pl.coalesce([
        pl.col('cpu_model').str.extract(r'([A-Za-z]+\s\d+)'),
        pl.col('cpu_model').str.extract(r'([\w]\d)')
    ]).alias('cpu_series')
)

In [ ]:
df

The user wants to round the result of the division, not the `storage_gb` column before division. The `round()` method should be applied to the entire expression result.



In [ ]:
df = df.with_columns(
    (pl.col('price') / pl.col('storage_gb')).round(2).alias('price_per_gb_storage')
)


In [ ]:
df

In [ ]:
df.collect_schema()

In [ ]:
df.select(
    pl.col('price', 'device_type')
).group_by(
    pl.col('device_type')
).agg(
    pl.sum('price')
)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df.to_pandas(), x='price', bins=30, kde=True)
plt.title('Distribution of Device Prices')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')
plt.show()

price_mean = df.select(pl.col('price').mean()).item()
price_median = df.select(pl.col('price').median()).item()

print(f"Mean Price: ${price_mean:.2f}")
print(f"Median Price: ${price_median:.2f}")
print(
    f"\nThe distribution is right-skewed because the mean (${price_mean:.2f}) is greater than the median (${price_median:.2f}).")
print(
    "This indicates that there are some expensive devices pulling the average up, with the tail extending to the right.")


In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(df.to_pandas(), x='price', bins=30, kde=True)
plt.title('Distribution of Device Prices')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')
plt.show()

price_mean = df.select(pl.col('price').mean()).item()
price_median = df.select(pl.col('price').median()).item()

print(f"Mean Price: ${price_mean:.2f}")
print(f"Median Price: ${price_median:.2f}")
print(
    f"\nThe distribution is right-skewed because the mean (${price_mean:.2f}) is greater than the median (${price_median:.2f}).")
print(
    "This indicates that there are some expensive devices pulling the average up, with the tail extending to the right.")

In [ ]:
top_5_brand = df.select(
    pl.col('brand')
).group_by(
    'brand'
).len().sort(
    by='len', descending=True
).head(5)

top_5_brand

In [ ]:
fig = px.bar(
    top_5_brand,
    x='brand',
    y='len'
)

fig.update_layout(
    title='Top 5 Brands by Number of Devices',
    width=800,
    height=600
)

fig

In [ ]:
os_counting = df.select(
    pl.col('os')
).group_by(
    'os'
).len().sort(
    by='len', descending=True
)
os_counting

In [ ]:
fig = px.bar(
    data_frame=os_counting
,
    x='os',
    y='len'
)

fig.update_layout(
    width=800,
    height=600,
)

fig.show()

In [ ]:
ram_price = df.select(
    pl.col('ram_gb', 'price')
)

ram_price.corr()

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(
    ram_price['ram_gb'],  # X-axis: RAM (GB)
    ram_price['price'],  # Y-axis: Price ($)
    c=ram_price['ram_gb'],
    cmap='viridis',
    alpha=0.8,
    edgecolors='none'
)
plt.title('RAM (GB) vs Price ($)')
plt.xlabel('RAM (GB)')
plt.ylabel('Price ($)')
plt.grid(False)

plt.show()


In [ ]:
df.select(
    pl.col('device_type', 'price')
).filter(
    (pl.col('device_type') == 'Desktop') |
    (pl.col('device_type') == 'Laptop')
).group_by(
    pl.col('device_type')
).agg(
    pl.mean('price').alias('price_mean'),
    pl.median('price').alias('price_median'),
    pl.sum('price').alias('price_sum'),
)

In [ ]:
cpu_tier_price = df.select(
    pl.col('cpu_tier', 'price')
).group_by(
    'cpu_tier'
).agg(
    pl.mean('price').alias('price_mean'),
)

cpu_tier_price

In [ ]:
fig = px.bar(
    data_frame=cpu_tier_price,
    x='cpu_tier',
    y='price_mean'
)

fig.update_layout(
    width=800,
    height=600,
)

fig.show()

In [ ]:
corr_between_cou_ghz_price = df.select(
    pl.col('cpu_cores', 'cpu_boost_ghz', 'price')
)

In [ ]:
fig = px.scatter(
    data_frame=corr_between_cou_ghz_price,
    x='cpu_cores',
    y='cpu_boost_ghz',
)

fig.update_layout(
    width=800,
    height=600,
    title='Correlation between CPU Cores',
)

fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df.to_pandas(),
    x='cpu_cores',
    y='cpu_boost_ghz',
    size='price',
    color='price',
    color_continuous_scale='Viridis',
    title='Relationship between CPU Cores, CPU Boost GHz, and Price',
    labels={
        'cpu_cores': 'CPU Cores',
        'cpu_boost_ghz': 'CPU Boost (GHz)',
        'price': 'Price ($)'
    },
    hover_data=['price']

)

fig.update_layout(
    width=1000,
    height=700
)

fig.show()


In [ ]:
fig = px.scatter(
    data_frame=corr_between_cou_ghz_price.to_pandas(),
    x='cpu_cores',
    y='cpu_boost_ghz',
    size='price',
    color='price',
    color_continuous_scale='Viridis',
    title='Relationship between CPU Cores, CPU Boost GHz, and Price',
    labels=[{
        'cpu_cores': 'CPU Cores',
        'cpu_boost_ghz': 'CPU Boost (GHz)',
        'price': 'Price ($)'
    }]
)
fig.update_layout(
    width=1000,
    height=700
)


fig.show()

In [ ]:
df.select(
    pl.col('storage_type', 'price_per_gb_storage')
).group_by(
    'storage_type'
).agg(
    pl.col('price_per_gb_storage').mean().alias('avg_price_per_gb'),
    pl.col('price_per_gb_storage').median().alias('median_price_per_gb'),
    pl.sum('price_per_gb_storage').alias('total_price_per_gb')
)

In [ ]:
df.select(
    pl.col('storage_type', 'price_per_gb_storage')
).group_by(
    'storage_type'
).agg(
    pl.col('price_per_gb_storage').mean().alias('avg_price_per_gb')
).sort(
    'avg_price_per_gb'
)


In [ ]:
df.filter(
    (pl.col('gpu_tier') >= 4) &
    (pl.col('price').is_between(2000, 3000))
).select(
    pl.mean('ram_gb').alias('ram_gb_mean'),
    pl.mean('vram_gb').alias('vram_gb_mean'),
    pl.mean('cpu_cores').alias('cpu_cores_mean'),
    pl.mean('price').alias('price_mean')
)

In [ ]:
df.filter(
    (pl.col('gpu_tier') >= 4) &
    (pl.col('price').is_between(2000, 3000))
).select(
    pl.col('ram_gb').mean().alias('avg_ram_gb'),
    pl.col('vram_gb').mean().alias('avg_vram_gb'),
    pl.col('cpu_cores').mean().alias('avg_cpu_cores')
)

In [ ]:
df.collect_schema()

In [ ]:
df.select(
    pl.col('warranty_months', 'price')
).group_by(
    pl.col('warranty_months')
).agg(
    pl.col('price').mean().alias('avg_prices'),
    pl.col('price').median().alias('median_prices'),
    pl.count('warranty_months').alias('count')
).sort(
    by='warranty_months'
)

In [ ]:
# Calculate correlation between warranty_months and price
correlation = df.select(
    pl.corr('warranty_months', 'price').alias('correlation')
).item()

print(f"Correlation between warranty_months and price: {correlation:.4f}")

# Create a scatter plot to visualize the relationship
fig = px.scatter(
    data_frame=df.to_pandas(),
    x='warranty_months',
    y='price',
    trendline='ols',
    title='Warranty Months vs Price',
    labels={
        'warranty_months': 'Warranty (Months)',
        'price': 'Price ($)'
    },
    opacity=0.6
)

fig.update_layout(
    width=1000,
    height=600
)

fig.show()

# Interpret the results
if correlation > 0.3:
    print(f"\n✓ Hypothesis SUPPORTED: Strong positive correlation ({correlation:.4f})")
    print("Devices with longer warranties tend to be more expensive.")
elif correlation > 0.1:
    print(f"\n~ Hypothesis PARTIALLY SUPPORTED: Weak positive correlation ({correlation:.4f})")
    print("There is a slight tendency for longer warranties to be associated with higher prices.")
elif correlation > -0.1:
    print(f"\n✗ Hypothesis DISPROVED: No significant correlation ({correlation:.4f})")
    print("Warranty length has little to no relationship with price.")
else:
    print(f"\n✗ Hypothesis DISPROVED: Negative correlation ({correlation:.4f})")
    print("Devices with longer warranties tend to be less expensive.")



## Summary of Task
Identify price outliers within each device_type group using the IQR method and create visualizations to show these outliers. First create a box plot grouped by device_type, then programmatically identify specific models that exceed the upper outlier threshold (Q3 + 1.5*IQR).



In [ ]:
# Create box plot to visualize price outliers by device_type
fig = px.box(
    data_frame=df.to_pandas(),
    x='device_type',
    y='price',
    title='Price Distribution by Device Type (with Outliers)',
    labels={
        'device_type': 'Device Type',
        'price': 'Price ($)'
    },
    points='outliers'
)

fig.update_layout(
    width=1000,
    height=600
)

fig.show()

# Identify outliers programmatically using IQR method
outliers = df.group_by('device_type').agg([
    pl.col('price').quantile(0.25).alias('Q1'),
    pl.col('price').quantile(0.75).alias('Q3')
]).with_columns(
    (pl.col('Q3') - pl.col('Q1')).alias('IQR')
).with_columns(
    (pl.col('Q3') + 1.5 * pl.col('IQR')).alias('upper_limit')
)

# Join back to original dataframe and filter for outliers
outlier_devices = df.join(
    outliers,
    on='device_type',
    how='left'
).filter(
    pl.col('price') > pl.col('upper_limit')
).select(
    pl.col('device_type', 'brand', 'model', 'price', 'upper_limit')
).sort(
    by=['device_type', 'price'],
    descending=[False, True]
)

print(f"\nTotal number of outliers found: {outlier_devices.height}")
print("\nTop 10 most expensive outliers:")
outlier_devices.head(10)

In [ ]:
outliers = df.group_by('device_type').agg([
    pl.col('price').quantile(0.25).alias('Q1'),
    pl.col('price').quantile(0.75).alias('Q3')
]).with_columns(
    (pl.col('Q3') - pl.col('Q1')).alias('IQR')
).with_columns(
    (pl.col('Q3') + 1.5 * pl.col('IQR')).alias('upper_limit')
)

outliers

In [ ]:
df.join(
    outliers,
    on='device_type',
    how='left'
).filter(
    pl.col('price') > pl.col('upper_limit')
).select(
    pl.col('device_type', 'brand', 'model', 'price', 'upper_limit')
).sort(
    by=['device_type', 'price'],
    descending=[False, True]
)

In [ ]:
df.collect_schema()

In [ ]:
df.filter(
    (pl.col('weight_kg') < 1.5) &
    (pl.col('cpu_tier') >= 4) &
    (pl.col('device_type') == 'Laptop')
)

In [ ]:
df.with_columns(
    (pl.col('cpu_tier') * 0.5 + pl.col('gpu_tier') * 0.3 + (pl.col('ram_gb') / 32) * 0.2).alias('performance_score')
)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

string_columns = df.select(
    cs.string()
)

string_columns.select([
    pl.col(col).n_unique().alias(f'{col}_unique') for col in string_columns.columns
])

In [ ]:
unique_str_columns = string_columns.select(
    cs.exclude('model')
)

numeric_columns = df.select(
    cs.numeric()
)

In [ ]:
df.to_dummies(drop_first=True, drop_nulls=True)